ATTENTION: This notebook is based on old mock data. It is in this git repository for reference. However this material may be outdated for future users who are reading this.

# Failed Batch Transform Job

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os
import sys
import time
import json
from sklearn import metrics
from IPython.display import display
from time import strftime, gmtime
import sagemaker # SageMaker Python SDK
from sagemaker.predictor import csv_serializer

In [2]:
bucket3 = 's3://ml-analytics-data-2022/'
prefix3 = 'Batch-input/output_1658951765/part-00000-f763dbe5-455a-44dc-ab9c-be4c367d3f78-c000.csv'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

print('Hi')

Hi


In [20]:

test_new = 's3://ml-analytics-data-2022/Batch-input/output_1658951765/part-00000-f763dbe5-455a-44dc-ab9c-be4c367d3f78-c000.csv'
#test_new = 's3://{}/{}'.format(bucket3, prefix3)

test_df_new = pd.read_csv(test_new)

In [21]:
test_df_new

,hircand_Role,hircand_Type,hircand_Level,hircand_Preferred_Time,hircand_Preferred_Day,hiremp_Employee_Alias,hiremp_Interviewer_Name,hiremp_Role,hiremp_Type,hiremp_Level,hiremp_Preferred_Time,hiremp_Preferred_Day,emp_alias,emp_name,emp_skill,emp_skill_rating
0,Data Scientist,Internship,L6,13:00,Friday,hlowles7,Hatti Lowles,Program Manager,Internship,L4,9:00,Wednesday,hlowles7,Hatti Lowles,Compute and HPC - EC2 (Elastic Compute Cloud),5
1,Technical Account Manager,Contract,L9,12:00,Friday,sdelmontib,Stacia Delmonti,Machine Learning Engineer,Full-time,L8,13:00,Tuesday,sdelmontib,Stacia Delmonti,ML/AI - Forecast,5
2,Data Scientist,Full-time,L5,8:00,Thursday,bgreshamh,Bronson Gresham,Delivery Practice Manager,Part-time,L4,15:00,Friday,bgreshamh,Bronson Gresham,Analytics - Lake Formation,1
3,Software Engineer,Internship,L4,8:00,Monday,zellingsi,Zach Ellings,Data Engineer,Contract,L6,15:00,Thursday,zellingsi,Zach Ellings,Database - RDS SQL Server,1
4,Cloud Architect,Full-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Supply Chain Manager,Part-time,L6,16:00,Thursday,cingyont,Carlina Ingyon,Database - RDS SQL Server,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Solutions Architect,Part-time,L9,15:00,Thursday,eplimmerr7,Ernestus Plimmer,Data Scientist,Contract,L6,17:00,Monday,eplimmerr7,Ernestus Plimmer,Mobile - Pinpoint,2
196,Cloud Architect,Contract,L7,12:00,Wednesday,aduddellra,Agnola Duddell,Financial Analyst,Contract,L9,17:00,Monday,aduddellra,Agnola Duddell,Database - RDS SQL Server,1
197,Cloud Architect,Contract,L6,10:00,Friday,rsticklesrd,Raphaela Stickles,Software Development Engineer,Internship,L4,13:00,Wednesday,rsticklesrd,Raphaela Stickles,Analytics - Lake Formation,1
198,Professional Services Consultant,Full-time,L5,16:00,Tuesday,afridayrg,Agathe Friday,Product Manager,Full-time,L9,8:00,Wednesday,afridayrg,Agathe Friday,Analytics - Lake Formation,1


In [8]:
test_data_batch = test_df_new.iloc[:, :-1]
test_data_batch # delete the target column
test_data_batch.to_csv('test_data_Batch.csv',header=False, index = False)
# upload to S3
#boto3.Session().resource('s3').Bucket(bucket3).Object(os.path.join(prefix3, 'test_data_Batch.csv')).upload_file('test_data_Batch.csv')

In [44]:
s3_batch_input = 's3://ml-analytics-data-2022/Batch-input/output_1658790247/part-00000-051e3a7f-69fe-4cbc-acd2-1ab25525cbac-c000.csv' # test data used for prediction
s3_batch_output = 's3://ml-analytics-data-2022/Batch-transform-output/test_data_Batch.csv' # specify the location of batch output

In [58]:
! pip install xgboost

In [59]:
model_name = 'Master-training-S3Mx2N66hwx6zo6x-072-e81b59ce'

In [62]:

container = get_image_uri(boto3.Session().region_name, 'xgboost')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ValueError: Unsupported xgboost version: 1. You may need to upgrade your SDK version (pip install -U sagemaker) for newer xgboost versions. Supported xgboost version(s): 1, 0.90-1, 0.90-2, 1.0-1, 1.2-1, 1.2-2, 1.3-1, 1.5-1, latest.

In [51]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    base_job_name = model_name,
                                    output_path='s3://ml-analytics-data-2022/autopilot-outputs/',
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

NameError: name 'container' is not defined

In [49]:

transformer = xgb.transformer(
                          instance_count=1,
                          instance_type='ml.m4.xlarge',
                          output_path=s3_batch_output)

# calls that object's transform method to create a transform job
transformer.transform(data=s3_batch_input, data_type='S3Prefix', content_type='text/csv', split_type='Line')

transformer.wait()

NameError: name 'xgb' is not defined

In [14]:
from time import gmtime, strftime
batch_input = 's3://ml-analytics-data-2022/Batch-input/output_1658790247/part-00000-051e3a7f-69fe-4cbc-acd2-1ab25525cbac-c000.csv'#.format(bucket,prefix) # test data used for prediction
batch_output = 's3://ml-analytics-data-2022/Batch-transform-output/'#.format(bucket,prefix)
Modelname = 'Master-training-S3Mx2N66hwx6zo6x-072-e81b59ce' # the model name we already have
transformJobName = 'Batch-job'+ strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [15]:
client = boto3.client('sagemaker')

In [16]:
create_batch = client.create_transform_job(
    TransformJobName=transformJobName,
    ModelName=Modelname,
    MaxConcurrentTransforms=0,
    MaxPayloadInMB=6,
    BatchStrategy='MultiRecord',
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': batch_input 
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    },
    TransformOutput={
        'S3OutputPath': batch_output,
        'AssembleWith': 'Line'
    },
    TransformResources={
        'InstanceType': 'ml.m4.xlarge',
        'InstanceCount': 1
    }
    )

In [7]:
import json 
import boto3 
client = boto3.client('runtime.sagemaker')
 
data = {"instances": ['Data Scientist', 'Full-time' ,'L5','8:00','Thursday','bgreshamh','Bronson Gresham','Delivery Practice Manager','Part-time','L4','15:00','Friday','bgreshamh','Bronson Gresham','Analytics - Lake Formation','1',0,0,0,0,0,0]} 
response = client.invoke_endpoint(EndpointName='real-time-pred',
                                  Body=json.dumps(data), ContentType='text/csv')
response_body = response['Body'] 
print(response_body.read())

b'No,"[0.17653846740722656, 0.8234615325927734]"\n'


In [12]:
pip install shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
def predict(data, rows=1000):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    return np.fromstring(predictions[1:], sep=',')

In [10]:
import matplotlib.pyplot as plt

predictions=predict(test_data_batch.to_numpy()[:,1:])
plt.hist(predictions)
plt.show()

NameError: name 'xgb_predictor' is not defined

In [29]:
from sagemaker.transformer import Transformer

transformer = Transformer(
    model_name='Final-model4MQgfyyKqLdfUjXEJqOUY-001-8b7b5c97-aws-trial',
    instance_count=1,
    instance_type='ml.m5.large',
    strategy='SingleRecord',
    output_path='s3://ml-analytics-data-2022/Batch-transform-output/',
    base_transform_job_name='raw-batch',
    accept='application/json'
)

In [30]:
transformer.transform(
    's3://ml-analytics-data-2022/Batch-input/output_1658951765/',
    content_type='application_json',
    split_type='Line',
    join_source='Input',
    wait=False)

ClientError: An error occurred (ValidationException) when calling the CreateTransformJob operation: Could not find model "arn:aws:sagemaker:us-east-1:741154895315:model/final-model4mqgfyykqldfujxejqouy-001-8b7b5c97-aws-trial".

# Manual Batch Transform from AutoPilot

In [34]:
from sagemaker import AutoML

autopilot_experiment_name = "final-model"
automl = AutoML.attach(auto_ml_job_name=autopilot_experiment_name)

In [40]:
inference_response_keys = ['predicted_label', 'probability']
batch_input = 's3://ml-analytics-data-2022/autopilot-outputs/output_1658776079/part-00000-e11fc45b-08ec-493d-a91b-8c4e6e56ee2b-c000.csv'
batch_output = 's3://ml-analytics-data-2022/autopilot-outputs/output_1658776079/Final-model/transformed-data/dpp0/rpb/train/'

In [41]:
best_candidate = automl.describe_auto_ml_job()['BestCandidate']
best_candidate_name = best_candidate['CandidateName']

model = automl.create_model(name=best_candidate_name, candidate=best_candidate, inference_response_keys=inference_response_keys)

In [42]:
transformer = model.transformer(instance_count=1, 
				 instance_type='ml.m5.12xlarge',
				 assemble_with='Line',
				 output_path=batch_output)
				 
transformer.transform(data=batch_input,
                      split_type='Line',
		      content_type='text/csv',
		      wait=False)